In [23]:
import pandas as pd
import datasets
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
import warnings
import json
import os
import io
from datasets import load_dataset

In [24]:
# ========== SETTINGS ==========
# Configure which dataset to process
DATASET_FOLDER = '5411_iso_splits'  # Options: '5411_temporal_splits' or '5411_iso_splits'
COST_FILE = 'cost_type_id_18feb.csv'

# File naming pattern
if 'temporal' in DATASET_FOLDER:
    TRAIN_FILE = '5411_train_temporal.csv'
    VALIDATE_FILE = '5411_validate_temporal.csv'
    TEST_FILE = '5411_test_temporal.csv'
    OUTPUT_SUFFIX = '_with_costs'
elif 'iso' in DATASET_FOLDER:
    TRAIN_FILE = '5411_train_iso.csv'
    VALIDATE_FILE = '5411_validate_iso.csv'
    TEST_FILE = '5411_test_iso.csv'
    OUTPUT_SUFFIX = '_with_costs'
else:
    # Generic fallback
    TRAIN_FILE = '5411_train.csv'
    VALIDATE_FILE = '5411_validate.csv'
    TEST_FILE = '5411_test.csv'
    OUTPUT_SUFFIX = '_with_costs'

print(f"Configuration:")
print(f"  Dataset folder: {DATASET_FOLDER}")
print(f"  Cost file: {COST_FILE}")
print(f"  Train file: {TRAIN_FILE}")
print(f"  Validate file: {VALIDATE_FILE}")
print(f"  Test file: {TEST_FILE}")


Configuration:
  Dataset folder: 5411_iso_splits
  Cost file: cost_type_id_18feb.csv
  Train file: 5411_train_iso.csv
  Validate file: 5411_validate_iso.csv
  Test file: 5411_test_iso.csv


In [25]:
# Load the splits from the configured folder
df_train = pd.read_csv(f'{DATASET_FOLDER}/{TRAIN_FILE}')
df_validate = pd.read_csv(f'{DATASET_FOLDER}/{VALIDATE_FILE}')
df_test = pd.read_csv(f'{DATASET_FOLDER}/{TEST_FILE}')

print(f"Train set loaded: {len(df_train)} transactions, {df_train['merchant_id'].nunique()} unique merchants")
print(f"Validate set loaded: {len(df_validate)} transactions, {df_validate['merchant_id'].nunique()} unique merchants")
print(f"Test set loaded: {len(df_test)} transactions, {df_test['merchant_id'].nunique()} unique merchants")
print(f"\nTotal transactions: {len(df_train) + len(df_validate) + len(df_test)}")
print(f"\nTrain columns: {df_train.columns.tolist()}")


Train set loaded: 708 transactions, 327 unique merchants
Validate set loaded: 801 transactions, 362 unique merchants
Test set loaded: 653 transactions, 300 unique merchants

Total transactions: 2162

Train columns: ['transaction_id', 'date', 'amount', 'use_chip', 'merchant_id', 'mcc', 'errors', 'card_brand', 'card_type', 'mcc_description', 'year']


In [26]:
# Load the cost CSV file
df_cost = pd.read_csv(COST_FILE)

print(f"Cost data loaded from {COST_FILE}: {len(df_cost)} rows")
print(f"Columns: {df_cost.columns.tolist()}")
print(f"First few rows:")
df_cost.head()


Cost data loaded from cost_type_id_18feb.csv: 62 rows
Columns: ['cost_type_ID', 'card_network', 'card_brand', 'fee_program', 'min_transaction_amt', 'max_transaction_amt', 'mcc', 'card_fee_percent', 'card_fee_dollars', 'network_fee_percent', 'network_fee_dollars', 'subtotal_fee_percent', 'subtotal_fee_dollars', 'Unnamed: 13']
First few rows:


,cost_type_ID,card_network,card_brand,fee_program,min_transaction_amt,max_transaction_amt,mcc,card_fee_percent,card_fee_dollars,network_fee_percent,network_fee_dollars,subtotal_fee_percent,subtotal_fee_dollars,Unnamed: 13
0,1.0,Visa,Prepaid,Small Ticket Fee Program (All),0.000,5.000,NaN,1.60%,$0.05,0.13%,$0.02,1.73%,$0.07,NaN
1,2.0,Visa,Debit,Small Ticket Fee Program (All),0.000,5.000,NaN,0.05%,$0.21,0.13%,$0.02,0.18%,$0.23,NaN
2,3.0,Visa,Credit,Small Ticket Fee Program (All),0.000,1.818,NaN,0.00%,$0.04,0.14%,$0.02,0.14%,$0.06,NaN
3,4.0,Visa,Credit,Small Ticket Fee Program (All),1.818,5.000,NaN,2.20%,$0.00,0.14%,$0.02,2.34%,$0.02,NaN
4,5.0,Visa,Super Premium Credit,Small Ticket Fee Program (All),0.000,1.818,NaN,0.00%,$0.04,0.14%,$0.02,0.14%,$0.06,NaN


In [27]:
# Function to apply cost_type_ID to a dataframe
def apply_cost_type_id(df, cost_type_df):
    """Apply cost_type_ID to transaction dataframe based on cost rules
    
    Note: Column mapping between transaction data and cost data:
    - Transaction 'card_brand' (Visa/Mastercard) → Cost 'card_network'
    - Transaction 'card_type' (Credit/Debit/Prepaid) → Cost 'card_brand'
    """
    df = df.copy()
    
    # Ensure mcc is same type in both dataframes
    df['mcc'] = df['mcc'].astype(float)
    cost_type_df['mcc'] = pd.to_numeric(cost_type_df['mcc'], errors='coerce')
    
    # Filter to only card brands that exist in cost_type_df
    df_filtered = df[df['card_brand'].isin(['Visa', 'Mastercard'])].copy()
    
    # Separate rules
    cost_general = cost_type_df[cost_type_df['mcc'].isna()].copy()
    cost_specific = cost_type_df[cost_type_df['mcc'].notna()].copy()
    
    # Process small (<=5): general rules
    df_small = df_filtered[df_filtered['amount'] <= 5].merge(
        cost_general,
        left_on=['card_brand', 'card_type'],
        right_on=['card_network', 'card_brand'],
        how='left',
        suffixes=('', '_cost')
    )
    df_small = df_small[
        (df_small['amount'] >= df_small['min_transaction_amt']) &
        (df_small['amount'] <= df_small['max_transaction_amt']) &
        (df_small['cost_type_ID'].notna())
    ]
    df_small = df_small.sort_values('transaction_id').drop_duplicates('transaction_id', keep='first')
    
    # Process large (>5): mcc-specific rules
    df_large = df_filtered[df_filtered['amount'] > 5].merge(
        cost_specific,
        left_on=['card_brand', 'card_type', 'mcc'],
        right_on=['card_network', 'card_brand', 'mcc'],
        how='left',
        suffixes=('', '_cost')
    )
    df_large = df_large[
        (df_large['amount'] >= df_large['min_transaction_amt']) &
        (df_large['amount'] <= df_large['max_transaction_amt']) &
        (df_large['cost_type_ID'].notna())
    ]
    df_large = df_large.sort_values('transaction_id').drop_duplicates('transaction_id', keep='first')
    
    # Combine results and map back to original df
    result_map = pd.concat([
        df_small[['transaction_id', 'cost_type_ID']],
        df_large[['transaction_id', 'cost_type_ID']]
    ]).set_index('transaction_id')['cost_type_ID']
    
    # Drop old cost_type_ID if exists
    if 'cost_type_ID' in df.columns:
        df = df.drop('cost_type_ID', axis=1)
    
    df['cost_type_ID'] = df['transaction_id'].map(result_map)
    
    return df

# Apply cost_type_ID to all three datasets
df_train = apply_cost_type_id(df_train, df_cost)
df_validate = apply_cost_type_id(df_validate, df_cost)
df_test = apply_cost_type_id(df_test, df_cost)

print("=== TRAIN SET ===")
print(f"Total transactions: {len(df_train)}")
print(f"Rows with cost_type_ID: {df_train['cost_type_ID'].notna().sum()}")
print(f"Rows without match: {df_train['cost_type_ID'].isna().sum()}")

print("\n=== VALIDATE SET ===")
print(f"Total transactions: {len(df_validate)}")
print(f"Rows with cost_type_ID: {df_validate['cost_type_ID'].notna().sum()}")
print(f"Rows without match: {df_validate['cost_type_ID'].isna().sum()}")

print("\n=== TEST SET ===")
print(f"Total transactions: {len(df_test)}")
print(f"Rows with cost_type_ID: {df_test['cost_type_ID'].notna().sum()}")
print(f"Rows without match: {df_test['cost_type_ID'].isna().sum()}")

=== TRAIN SET ===
Total transactions: 708
Rows with cost_type_ID: 661
Rows without match: 47

=== VALIDATE SET ===
Total transactions: 801
Rows with cost_type_ID: 743
Rows without match: 58

=== TEST SET ===
Total transactions: 653
Rows with cost_type_ID: 604
Rows without match: 49


In [28]:
# Prepare lookup arrays indexed by cost_type_ID
df_cost['subtotal_fee_percent_clean'] = df_cost['subtotal_fee_percent'].str.rstrip('%').astype(float) / 100
df_cost['subtotal_fee_dollars_clean'] = df_cost['subtotal_fee_dollars'].str.lstrip('$').astype(float)

# Create dictionaries for O(1) lookup
fee_dollars_map = dict(zip(df_cost['cost_type_ID'], df_cost['subtotal_fee_dollars_clean']))
fee_percent_map = dict(zip(df_cost['cost_type_ID'], df_cost['subtotal_fee_percent_clean']))

# Apply proc_cost calculation to all three datasets
df_train['proc_cost'] = df_train['cost_type_ID'].map(fee_dollars_map) + (df_train['cost_type_ID'].map(fee_percent_map) * df_train['amount'])
df_validate['proc_cost'] = df_validate['cost_type_ID'].map(fee_dollars_map) + (df_validate['cost_type_ID'].map(fee_percent_map) * df_validate['amount'])
df_test['proc_cost'] = df_test['cost_type_ID'].map(fee_dollars_map) + (df_test['cost_type_ID'].map(fee_percent_map) * df_test['amount'])

print("=== TRAIN SET ===")
print(f"✓ proc_cost calculated for {df_train['proc_cost'].notna().sum():,} transactions")

print("\n=== VALIDATE SET ===")
print(f"✓ proc_cost calculated for {df_validate['proc_cost'].notna().sum():,} transactions")

print("\n=== TEST SET ===")
print(f"✓ proc_cost calculated for {df_test['proc_cost'].notna().sum():,} transactions")

=== TRAIN SET ===
✓ proc_cost calculated for 661 transactions

=== VALIDATE SET ===
✓ proc_cost calculated for 743 transactions

=== TEST SET ===
✓ proc_cost calculated for 604 transactions


In [29]:
# Save preview of costed transactions from each dataset
df_train.head(25).to_csv('preview_train_costed.csv', index=False)
df_validate.head(25).to_csv('preview_validate_costed.csv', index=False)
df_test.head(25).to_csv('preview_test_costed.csv', index=False)

print("Preview files saved:")
print("- preview_train_costed.csv")
print("- preview_validate_costed.csv")
print("- preview_test_costed.csv")

Preview files saved:
- preview_train_costed.csv
- preview_validate_costed.csv
- preview_test_costed.csv


In [30]:
# Save the costed datasets back to the same folder
train_output = TRAIN_FILE.replace('.csv', f'{OUTPUT_SUFFIX}.csv')
validate_output = VALIDATE_FILE.replace('.csv', f'{OUTPUT_SUFFIX}.csv')
test_output = TEST_FILE.replace('.csv', f'{OUTPUT_SUFFIX}.csv')

df_train.to_csv(f'{DATASET_FOLDER}/{train_output}', index=False)
df_validate.to_csv(f'{DATASET_FOLDER}/{validate_output}', index=False)
df_test.to_csv(f'{DATASET_FOLDER}/{test_output}', index=False)

print(f"Datasets with costs saved to {DATASET_FOLDER}/ directory:")
print(f"- {train_output}: {len(df_train)} rows")
print(f"- {validate_output}: {len(df_validate)} rows")
print(f"- {test_output}: {len(df_test)} rows")
print(f"\nColumns in saved files: {df_train.columns.tolist()}")


Datasets with costs saved to 5411_iso_splits/ directory:
- 5411_train_iso_with_costs.csv: 708 rows
- 5411_validate_iso_with_costs.csv: 801 rows
- 5411_test_iso_with_costs.csv: 653 rows

Columns in saved files: ['transaction_id', 'date', 'amount', 'use_chip', 'merchant_id', 'mcc', 'errors', 'card_brand', 'card_type', 'mcc_description', 'year', 'cost_type_ID', 'proc_cost']


# Diagnostics

In [31]:
# Check columns in both dataframes
print("=== Transaction Data Columns ===")
print(df_train.columns.tolist())
print("\n=== Cost Data Columns ===")
print(df_cost.columns.tolist())
print("\n=== Transaction Data Sample ===")
print(df_train.head(3))
print("\n=== Cost Data Sample ===")
print(df_cost.head(3))


=== Transaction Data Columns ===
['transaction_id', 'date', 'amount', 'use_chip', 'merchant_id', 'mcc', 'errors', 'card_brand', 'card_type', 'mcc_description', 'year', 'cost_type_ID', 'proc_cost']

=== Cost Data Columns ===
['cost_type_ID', 'card_network', 'card_brand', 'fee_program', 'min_transaction_amt', 'max_transaction_amt', 'mcc', 'card_fee_percent', 'card_fee_dollars', 'network_fee_percent', 'network_fee_dollars', 'subtotal_fee_percent', 'subtotal_fee_dollars', 'Unnamed: 13', 'subtotal_fee_percent_clean', 'subtotal_fee_dollars_clean']

=== Transaction Data Sample ===
   transaction_id                 date  amount          use_chip  merchant_id  \
0        18882165  2017-01-01 07:24:00    0.24  Chip Transaction        13456   
1        18882220  2017-01-01 07:34:00    0.18  Chip Transaction        13456   
2        18896821  2017-01-04 09:05:00    9.33  Chip Transaction        53657   

      mcc errors  card_brand card_type               mcc_description  year  \
0  5411.0    NaN

In [32]:
# Analyze unmatched transactions in train set
unmatched_train = df_train[df_train['cost_type_ID'].isna()].copy()

print("=== ANALYSIS OF UNMATCHED TRANSACTIONS (TRAIN SET) ===\n")
print(f"Total unmatched: {len(unmatched_train)}")

print("\n1. Card Brand Distribution (unmatched):")
print(unmatched_train['card_brand'].value_counts())

print("\n2. Card Type Distribution (unmatched):")
print(unmatched_train['card_type'].value_counts())

print("\n3. Amount Statistics (unmatched):")
print(f"   Min: ${unmatched_train['amount'].min():.2f}")
print(f"   Max: ${unmatched_train['amount'].max():.2f}")
print(f"   Mean: ${unmatched_train['amount'].mean():.2f}")
print(f"   Negative amounts: {(unmatched_train['amount'] < 0).sum()}")

print("\n4. Sample of unmatched transactions:")
print(unmatched_train[['transaction_id', 'card_brand', 'card_type', 'amount', 'mcc']].head(10))

# Check what card brands and types are in the cost data
print("\n=== COST DATA COVERAGE ===")
print("\nCard networks in cost data:")
print(df_cost['card_network'].unique())
print("\nCard brands in cost data:")
print(df_cost['card_brand'].unique())

# For Visa/Mastercard transactions that are still unmatched
visa_mc_unmatched = unmatched_train[unmatched_train['card_brand'].isin(['Visa', 'Mastercard'])]
if len(visa_mc_unmatched) > 0:
    print(f"\n5. Visa/Mastercard transactions that didn't match: {len(visa_mc_unmatched)}")
    print("   Card type distribution:")
    print(visa_mc_unmatched['card_type'].value_counts())
    print("\n   Sample:")
    print(visa_mc_unmatched[['transaction_id', 'card_brand', 'card_type', 'amount', 'mcc']].head(10))


=== ANALYSIS OF UNMATCHED TRANSACTIONS (TRAIN SET) ===

Total unmatched: 47

1. Card Brand Distribution (unmatched):
card_brand
Amex        35
Discover    12
Name: count, dtype: int64

2. Card Type Distribution (unmatched):
card_type
Credit    47
Name: count, dtype: int64

3. Amount Statistics (unmatched):
   Min: $0.18
   Max: $175.78
   Mean: $24.46
   Negative amounts: 0

4. Sample of unmatched transactions:
     transaction_id card_brand card_type  amount     mcc
0          18882165   Discover    Credit    0.24  5411.0
1          18882220   Discover    Credit    0.18  5411.0
9          18921413       Amex    Credit   15.77  5411.0
49         19080931       Amex    Credit    2.08  5411.0
71         19114177       Amex    Credit   71.03  5411.0
91         19147263       Amex    Credit   10.06  5411.0
124        19295878       Amex    Credit   23.73  5411.0
135        19321294       Amex    Credit    9.47  5411.0
140        19340588       Amex    Credit    8.14  5411.0
142        1934